# 1. Load packages

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [14]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd## 1.1 Define output folder

output_main_folder = r'L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2'
output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

1972
Use existing analysis folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\analysis.
Use existing figure folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\figures.


In [15]:
import seaborn as sns
import scanpy as sc
import numpy as np

## 1.1 Define output folder

In [16]:
output_main_folder = r'L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables'
output_sub_folder = os.path.join(output_main_folder, 'Primary')
#output_figure_folder = os.path.join(output_main_folder, 'figures')

make_output_folder = True

if output_main_folder and not os.path.exists(output_main_folder):
    os.makedirs(output_main_folder)
    print(f'Generating output folder: {output_main_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing output folder: {output_main_folder}.')

    
if output_main_folder and not os.path.exists(output_sub_folder):
    os.makedirs(output_sub_folder)
    print(f'Generating output subfolder: {output_sub_folder}.')
elif os.path.exists(output_sub_folder):
    print(f'Use existing output subfolder: {output_sub_folder}.')

Use existing output folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables.
Use existing output subfolder: L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables\Primary.


# 2. Load library folder

In [17]:
library_main_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries'

merfish_folder = os.path.join(library_main_folder,'Existing_MERFISH')

ctp11_folder = os.path.join(library_main_folder, 'CTP-11_brain')
ctp12_folder = os.path.join(library_main_folder, 'CTP-12_mop_markers')
ctp13_folder = os.path.join(library_main_folder, 'CTP-13_brain-super-enhancers')


In [18]:
merfish_fname = os.path.join(merfish_folder, 'M1_meng.fasta')
ctp11_fname = os.path.join(ctp11_folder, 'final_pool_probes.fasta')
ctp12_fname = os.path.join(ctp12_folder, 'merged_CTP12.fasta')
ctp13_fname = os.path.join(ctp13_folder, 'final_pool_probes_combinatory.fasta')


## convert each library

In [27]:
from Bio import SeqIO

probes_df_dict = {}
df_savename_list = ['RNA MERFISH probes', 
                    'Genome chromatin probes', 
                    'Marker chromatin probes', 
                    'Super-enhancer chromatin probes']

for final_pb_filename, df_savename in zip([merfish_fname, ctp11_fname, ctp12_fname, ctp13_fname],df_savename_list):
    print(f'Process {df_savename}')
    # export fasta info
    probes_dict = {'name':[],'sequence':[]}
    with open(final_pb_filename, 'r') as handle:
        for record in SeqIO.parse(handle, "fasta"):
            probes_dict['name'].append(record.description)
            probes_dict['sequence'].append(str(record.seq))
            
    # convert df 
    probes_df = pd.DataFrame(probes_dict)
    if df_savename == 'RNA MERFISH probes': # remove sequenctial from MERFISH
        num_remove = len(probes_df['name'].str.contains('_sequential'))
        probes_df = probes_df[~probes_df['name'].str.contains('_sequential')]
        print(f'Removed {num_remove} probes for {df_savename}')
        
    if df_savename == 'Marker chromatin probes': # remove CTP11 purchase from CTP12
        num_remove = len(probes_df['name'].str.contains('CTP11_MO'))
        probes_df = probes_df[~probes_df['name'].str.contains('CTP11_MO')]
        print(f'Removed {num_remove} probes for {df_savename}')
    
    probes_df_dict[df_savename] = probes_df
    print(f'Kept {len(probes_df)} probes for {df_savename}')
    print(f'=================')
    

Process RNA MERFISH probes
Removed 22734 probes for RNA MERFISH probes
Kept 22005 probes for RNA MERFISH probes
Process Genome chromatin probes
Kept 173891 probes for Genome chromatin probes
Process Marker chromatin probes
Removed 5620 probes for Marker chromatin probes
Kept 3360 probes for Marker chromatin probes
Process Super-enhancer chromatin probes
Kept 115776 probes for Super-enhancer chromatin probes


In [28]:
probes_df_dict['Marker chromatin probes']

,name,sequence
2260,loc_18:15400982-15420982_gene_Aqp4_pb_73_pos_5...,CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGTTCCAT...
2261,loc_18:15400982-15420982_gene_Aqp4_pb_74_pos_5...,CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGCAGCCT...
2262,loc_18:15400982-15420982_gene_Aqp4_pb_75_pos_5...,CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGACATCT...
2263,loc_18:15400982-15420982_gene_Aqp4_pb_76_pos_5...,CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGGGCCCA...
2264,loc_18:15400982-15420982_gene_Aqp4_pb_77_pos_5...,CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGTTCACC...
...,...,...
5615,loc_11:78489091-78509091_gene_Vtn_pb_187_pos_1...,CGCAAACTGGTGCGGAAGGCAAGTCCGCGCAGGAACGTGAAACTCA...
5616,loc_11:78489091-78509091_gene_Vtn_pb_188_pos_1...,CGCAAACTGGTGCGGAAGGCAAGTCCGCGCAGGAACGTGAAACAGA...
5617,loc_11:78489091-78509091_gene_Vtn_pb_189_pos_1...,CGCAAACTGGTGCGGAAGGCAAGTCCGCGCAGGAACGTGAGAGAGA...
5618,loc_11:78489091-78509091_gene_Vtn_pb_190_pos_1...,CGCAAACTGGTGCGGAAGGCAAGTCCGCGCAGGAACGTGAAGCAGA...


# save each library

In [29]:
for _name, _df in probes_df_dict.items():
    csv_fname = os.path.join(output_sub_folder, f'{_name}.csv')
    if not os.path.exists(csv_fname):
        print(f'Save {_name}')
        _df.to_csv(csv_fname, index=False)
    else:
        print(f'Skip {_name}')

Save RNA MERFISH probes
Save Genome chromatin probes
Save Marker chromatin probes
Save Super-enhancer chromatin probes


## save as seperate excel sheet

In [30]:
excel_fname = os.path.join(output_main_folder, f'Table_S1_MOp_primary_oligopropes.xlsx')

if not os.path.exists(excel_fname):
    print(f'Save {excel_fname}')
    with pd.ExcelWriter(excel_fname) as writer:  
        for _name, _df in probes_df_dict.items():
            _df.to_excel(writer, sheet_name=_name, index=False)
        
else:
    print(f'Skip {excel_fname}')
        

Save L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables\Table_1_MOp_primary_oligopropes.xlsx
